In [79]:
# 7111056119-02-basic RGB color transfer
import os
from pathlib import Path
import cv2
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [80]:
Path("result").mkdir(exist_ok=True)
Path("coltra").mkdir(exist_ok=True) # Output file

In [81]:
def readSourceImage(filepath:str):
    iim = cv2.imread(filename=filepath)
    # OpenCV 預設 BGR
    iim = cv2.cvtColor(iim, cv2.COLOR_BGR2RGB)
    return iim
    
# https://stackoverflow.com/questions/42406338/why-cv2-imwrite-changes-the-color-of-pics
def saveImage(filepath: str, im: cv2.Mat):
    im = cv2.cvtColor(im, cv2.COLOR_RGB2BGR)
    cv2.imwrite(filepath, im)
    ()

In [82]:
def colorChannelToStatistics(colorChannel: np.ndarray):
    uu, cc = np.unique(colorChannel, return_counts=True)
    ua = np.zeros(256, dtype=int)
    ua[uu] = cc
    aver = np.average(colorChannel)
    aver = np.around(aver, 2)
    std = np.std(colorChannel)
    std = np.around(std, 2)
    return (ua, aver, std)

def meanAndSTDToCSV(filename: str, mean, std):
    df = pd.DataFrame([mean, std], ["Mean", "STD"])
    df.to_csv("result/" + filename + "-mean-std.csv")
    ()

def hisToCSVAndPNG(filename:str, his):
    xx = np.array(his)
    df = pd.DataFrame(xx.transpose())
    df.to_csv("result/" + filename + "-his.csv")

    fig, ax = plt.subplots(3,1)
    ax[0].bar(range(256), his[0], color="red")
    ax[1].bar(range(256), his[1], color="green")
    ax[2].bar(range(256), his[2], color="blue")
    plt.savefig("result/" + filename + "_hist.png")
    ()

def doStatistics(filename: str, im: cv2.Mat):
    redHis, redAver, redStd = colorChannelToStatistics(im[:,:,0])
    greenHis, greenAver, greenStd = colorChannelToStatistics(im[:,:,1])
    blueHis, blueAver, blueStd = colorChannelToStatistics(im[:,:,2])

    meanAndSTDToCSV(filename, [redAver, greenAver, blueAver], [redStd, greenStd, blueStd])
    hisToCSVAndPNG(filename, [redHis, greenHis, blueHis])
    ()

In [83]:
def doColorTransfer(sourceChannel: np.ndarray, targetChannel: np.ndarray):
    sHis, sMean, sStd = colorChannelToStatistics(sourceChannel)
    tHis, tMean, tStd = colorChannelToStatistics(targetChannel)

    transfer = (tStd / sStd) * (sourceChannel - sMean) + tMean
    transfer = np.clip(transfer, 0, 255)
    return (sHis, sMean, sStd, tHis, tMean, tStd, transfer)
    ()

# target 圖片化為函數
# source 圖片與函數作用後，輸出至 colortransfer directory
# 其餘 his, csv 輸出至 result directory
# 轉換公式為 pptx 第 15 頁
def doStaff(sourcefile: str, targetfile: str, colortransferfile: str):
    ss = readSourceImage("source/" + sourcefile + ".png")
    tt = readSourceImage("ttarget/" + targetfile + ".png")

    sourceHis = []
    sourceMean = []
    sourceStd = []

    targetHis = []
    targetMean = []
    targetStd = []

    transfers = np.ndarray(ss.shape, dtype=ss.dtype)
    for i in range(3):
        (sHis, sMean, sStd, tHis, tMean, tStd, transfer) \
        = doColorTransfer(ss[:,:,i], tt[:,:,i])

        sourceHis.append(sHis)
        sourceMean.append(sMean)
        sourceStd.append(sStd)

        targetHis.append(tHis)
        targetMean.append(tMean)
        targetStd.append(tStd)

        transfers[:,:,i] = transfer
        ()

    # hisToCSVAndPNG(sourcefile, sourceHis)
    # meanAndSTDToCSV(sourcefile, sourceMean, sourceStd)

    # hisToCSVAndPNG(targetfile, targetHis)
    # meanAndSTDToCSV(targetfile, targetMean, targetStd)
    # print(transfers.shape)
    # doStatistics(colortransferfile, transfers)
    saveImage("coltra/" + colortransferfile + ".png", transfers)
    ()


In [ ]:
sourceImages = [
    "01_kodim06",
]

targetImages = [
    "01_kodim01",
]

colorTransferImages = [
    "01_kodim06_kodim01",
]

for i in range(len(sourceImages)):
    doStaff(sourceImages[i], targetImages[i], colorTransferImages[i])